In [2]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [1]:
df = pandas.read_csv("SMSSpamCollection", sep="\t", header=None, names=["label", "message"])

NameError: name 'pandas' is not defined